In [1]:
import math

In [2]:
class reacciones:
    def __init__(self, case, fcdead, Vx1, Fy1, Vz1, Vx2, Fy2, Vz2, Vx3, Fy3, Vz3):
        self.case = case  # combinacion de carga
        self.fcdead = fcdead #factor de carga de la carga muerta aplicado en las reacciones
        # dado solitario
        self.Vx1 = Vx1  # cortante x dado 1 [ton]
        self.Fy1 = Fy1  # fuerza vertical dado 1 [ton]
        self.Vz1 = Vz1  # cortante z dado 1 [ton]
        # dado 2
        self.Vx2 = Vx2  # cortante x dado 2 [ton]
        self.Fy2 = Fy2  # fuerza vertical dado 2 [ton]
        self.Vz2 = Vz2  # cortante z dado 2 [ton]
        # dado 3
        self.Vx3 = Vx3  # cortante x dado 3 [ton]
        self.Fy3 = Fy3  # fuerza vertical dado 3 [ton]
        self.Vz3 = Vz3  # cortante z dado 3 [ton]

    def peso_torre(self):
        return (self.Fy1 + self.Fy2 + self.Fy3)  / self.fcdead #Peso de la torre calculado de las reacciones [ton]

    def atributos(self):
        print("Combinación de carga: ",self.case)
        print("Dado 1 Vx = ",self.Vx1," [ton]")
        print("Dado 1 Fy = ",self.Fy1," [ton]")
        print("Dado 1 Vz = ",self.Vz1," [ton]")
        print("Dado 2 Vx = ",self.Vx2," [ton]")
        print("Dado 2 Fy = ",self.Fy2," [ton]")
        print("Dado 2 Vz = ",self.Vz2," [ton]")
        print("Dado 3 Vx = ",self.Vx3," [ton]")
        print("Dado 3 Fy = ",self.Fy3," [ton]")
        print("Dado 3 Vz = ",self.Vz3," [ton]")
           

In [3]:
class Concreto:
    def __init__(self, tipo, fc, gconc):
        self.tipo = tipo #Tipo de concreto
        self.fc = fc #f'c del concreto kg/cm2
        self.gconc = gconc #Peso volumetrico del concreto ton/m3

    def atributos(self):
        print("Concreto tipo ",self.tipo)
        print("Resistencia del concreto a 28 dias f'c = ",self.fc)
        print("Peso volumentrico del concreto ton/m3: ",self.gconc)

In [4]:
class Acero:
    def __init__(self, grado, Fy):
        self.grado = grado #Grado del acero de refuerzo principal
        self.Fy = Fy #Esfuerzo de fluencia acero de refuerzo principal

    def atributos(self):
        print("Acero grado ",self.grado)
        print("Esfuerzo de fluencia del acero: ",self.Fy)

In [5]:
class Suelo:
    def __init__(self, dfmin, grell):
        self.dfmin = dfmin #Profundidad minima de desplante
        self.grell = grell #Peso volumetrico del relleno ton/m3

In [6]:
class Zapata:

    def __init__(self, nombre, Bx, Bz, hl, Df, dx, dz, hd, sp, acero, concreto, suelo, reacciones ):
        self.nombre = nombre
        self.Bx = Bx #Ancho de losa en X [m]
        self.Bz = Bz #Ancho de losa en z [m]
        self.hl = hl #Espesor de la losa [m]
        self.Df = Df #Profundidad de desplante [m]
        self.dx = dx #Ancho de dado x
        self.dz = dz #Ancho de dado Z
        self.hd = hd #Altura que sobresale el dado sobre el terreno
        self.sp = sp #Separacion entre piernas
        self.acero = acero
        self.concreto = concreto
        self.suelo = suelo
        self.reacciones = reacciones

    def atributos(self):
        print(self.nombre,": ")
        print("Ancho x [m]: ",self.Bx)
        print("Ancho z [m]: ",self.Bz)
        print("Espesor [m]: ",self.hl)
        print("Profundidad de desplante [m]: ",self.Df)
        print("Ancho de dado x [m]: ",self.dx)
        print("Ancho de dado z [m]: ",self.dx)
        print("Altura dado sobre terreno [m]: ",self.hd)
        print("Separacion de piernas [m]: ",self.sp)

    #Calculo de volumenes y pesos de la zapata

    def vol_zap(self):
        return self.Bx * self.Bz * self.hl #Volumen de la zapata. [m3]
    
    def h_dado(self):
        return self.Df + self.hd - self.hl #Altura total de dado. [m]
    
    def vol_dado(self):
        return self.dx * self.dz * self.h_dado() #Volumen de 1 dado. [m3]
    
    def peso_dado(self):
        return self.vol_dado() * self.concreto.gconc #Peso de 1 dado. [ton]
    
    def peso_zap(self):
        return self.vol_zap() * self.concreto.gconc #Peso de la zapata. [ton]

    def vol_rell(self):
        return ((self.Bx * self.Bz) - (3 * (self.dx * self.dz))) * (self.Df - self.hl) #Volumen del relleno [m3]
    
    def peso_rell(self):
        return self.vol_rell() * self.suelo.grell #Peso del relleno [ton]
    
    #distancias de los elementos al borde frontal (dado solitario)

    def z_losa(self):
        return self.Bz / 2
    
    def spz(self):
        return self.sp * math.sin(math.radians(60)) #separacion de piernas en direccion z, proyeccion
    
    def z_dado1(self):
        return 0.5 * (self.Bz - self.spz())
    
    def z_dado2(self):
        return self.spz() + self.z_dado1()
    
    def z_rell(self):
        return ((self.Bx * self.Bz) * (self.Bz/2) - (self.dx * self.dz) * self.z_dado1() - (self.dx * self.dz) * self.z_dado2()) / ((self.Bx * self.Bz) - (self.dx * self.dz) - (self.dx * self.dz))
    
    def z_torre(self):
        return self.z_dado1() + ((2/3) * self.spz())

    #Calculo de momento resistente alrededor del eje frontal

    def Mrx1(self):
        return self.peso_zap() * self.z_losa() + self.peso_dado() * (self.z_dado1() + 2 * self.z_dado2()) + self.peso_rell() * self.z_rell() + self.reacciones.peso_torre() * self.z_torre()

    #Calculo de momento resistente alrededor de eje posterior

    def Mrx2(self):
        return self.peso_zap() * self.z_losa() + self.peso_dado() * ((self.Bz - self.z_dado1()) + 2 * (self.Bz - self.z_dado2())) + self.peso_rell() * (self.Bz - self.z_rell()) + self.reacciones.peso_torre() * (self.Bz - self.z_torre())

    #Calculo de momento actuante alrededor de eje 1
    
    #formula aplicable a las reacciones en el formato que las proporciona SITES
    
    def Max1(self):
        return -(self.reacciones.Fy1 - self.reacciones.peso_torre()/3) * self.z_dado1() - ((self.reacciones.Fy2 - self.reacciones.peso_torre()/3 + self.reacciones.Fy3 - self.reacciones.peso_torre()/3)) * self.z_dado2() + (self.reacciones.Vz1 + self.reacciones.Vz2 + self.reacciones.Vz3) * (self.Df + self.hd)

    #Calculo de momento actuante alrededor de eje 1
    
    #formula aplicable a las reacciones en el formato que las proporciona SITES

    def Max2(self):
        return (-self.reacciones.Fy1 + self.reacciones.peso_torre()/3) * (self.Bx - self.z_dado1()) - (self.reacciones.Fy2 + self.reacciones.Fy3 - 2 * self.reacciones.peso_torre()/3) * self.z_dado1() - (self.reacciones.Vz1 + self.reacciones.Vz2 + self.reacciones.Vz3) * (self.Df + self.hd)

    

### Reacciones de la torre

#### Combinacion de carga 16

In [7]:
load16 = reacciones("load 16",1.2, 0.007, 67.09, 5.32, -1.83, -28.47, 1.52, -1.85, -28.89, 1.51)
load16.atributos()

Combinación de carga:  load 16
Dado 1 Vx =  0.007  [ton]
Dado 1 Fy =  67.09  [ton]
Dado 1 Vz =  5.32  [ton]
Dado 2 Vx =  -1.83  [ton]
Dado 2 Fy =  -28.47  [ton]
Dado 2 Vz =  1.52  [ton]
Dado 3 Vx =  -1.85  [ton]
Dado 3 Fy =  -28.89  [ton]
Dado 3 Vz =  1.51  [ton]


#### Combinacion de carga 25

In [8]:
load25 = reacciones("load 25)",0.9, -0.007, -61.46, -4.93, -2.16, 34.17, -1.71, 2.19, 34.59, -1.71)
load25.atributos()

Combinación de carga:  load 25)
Dado 1 Vx =  -0.007  [ton]
Dado 1 Fy =  -61.46  [ton]
Dado 1 Vz =  -4.93  [ton]
Dado 2 Vx =  -2.16  [ton]
Dado 2 Fy =  34.17  [ton]
Dado 2 Vz =  -1.71  [ton]
Dado 3 Vx =  2.19  [ton]
Dado 3 Fy =  34.59  [ton]
Dado 3 Vz =  -1.71  [ton]


### Geometria de la cimentación

### Materiales

Concreto

In [9]:
fc280 = Concreto("Tipo 1",250, 2.2)
fc280.atributos()

Concreto tipo  Tipo 1
Resistencia del concreto a 28 dias f'c =  250
Peso volumentrico del concreto ton/m3:  2.2


Acero

In [10]:
AceroG42 = Acero("Grado 42",4200)
AceroG42.atributos()

Acero grado  Grado 42
Esfuerzo de fluencia del acero:  4200


In [12]:
suelo01 = Suelo(2.20,1.60)

In [13]:
zapata01 = Zapata("Zapata 01",6.00,6.00,0.55,2.20,0.60,0.60,0.30,4.68, AceroG42,fc280, suelo01, load16 )
zapata02 = Zapata("Zapata 01",6.00,6.00,0.55,2.20,0.60,0.60,0.30,4.68, AceroG42,fc280, suelo01, load25 )
zapata01.atributos()

Zapata 01 : 
Ancho x [m]:  6.0
Ancho z [m]:  6.0
Espesor [m]:  0.55
Profundidad de desplante [m]:  2.2
Ancho de dado x [m]:  0.6
Ancho de dado z [m]:  0.6
Altura dado sobre terreno [m]:  0.3
Separacion de piernas [m]:  4.68


In [ ]:








print(f"El volumen de la zapata es: ",zapata01.vol_zap(), " m³")
print(f"El peso de la zapata es: ",zapata01.peso_zap(), " ton")
print(f"El peso de la torre es: ",load16.peso_torre(), " ton")
print(f"El volumen del relleno es: ",zapata01.vol_rell(), " m3")
print(f"El peso del relleno es: ",zapata01.peso_rell(), " ton")
print(f"separacion: ",zapata01.sp, " m")
print(f"separacion: ",zapata01.z_dado1(), " m")
print(f"separacion: ",zapata01.z_torre(), " m")
print(f"Max1: ",zapata01.Max1(), " ton * m")
print(f"Max2: ",zapata02.Max2(), " ton * m")
print(f"Mrx1: ",zapata01.Mrx1(), " ton * m")
print(f"Mrx2: ",zapata02.Mrx2(), " ton * m")
print(f"0.9 Mrx1: ",0.9*zapata01.Mrx1(), " ton * m")
print(f"0.9 Mrx2: ",0.9*zapata02.Mrx2(), " ton * m")
print(f"Max1 / 0.9 Mrx1 = ",(zapata01.Max1())/(0.9*zapata01.Mrx1()))
print(f"Max2 / 0.9 Mrx2 = ",(zapata02.Max2())/(0.9*zapata02.Mrx2()))